In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

In [2]:
def parser(word):
    res = ['' for _ in range(14)]
    res[0] = word
    
    url = "https://fr.wiktionary.org/wiki/Conjugaison:français/" + word # url для страницы
    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html,'lxml')

    if soup.find('div', id=('mb0og1')) is None:
        div = soup.find('div', class_=('mw-parser-output'))
    else:
        if ((len(soup.find('div', id=('mb0og1')).find_all('table')) < 2) or 
            (word in ['décroitre', 'ressouvenir', 'souvenir'])):
            div = soup.find('div', id=('mb0og2'))
        else:
            div = soup.find('div', id=('mb0og1'))

    if div.find('span', id='Formes_usuelles') is not None:
        tables = div.find_all('table')[:-1]
    else:
        tables = div.find_all('table')

    if div.find('span', id=('Conjugaison_alternative')) is not None:
        tables = tables[:-3]

    if len(tables) > 22:
        tables = tables[-22:]

    trs = tables[1].find_all('tr')
    res[1] = trs[1].find_all('a')[2].find_all('span')[0].text.replace(".", "")[1:-1]
    res[1] = res[1].replace('s‿', "").replace('sə ', "")
    tds = trs[3].find_all('td')
    res[12] = tds[-2].text.replace("\n", "")
    res[13] = tds[-1].find_all('span')[0].text.replace(".", "").replace("\n", "")[1:-1]

    trs = tables[3].find_all('tr')
    tds = trs[1].find_all('td')
    res[2] = tds[1].text
    res[3] = tds[3].find_all('span')[0].text.replace(".", "")
    tds = trs[4].find_all('td')
    res[4] = tds[1].text
    res[5] = tds[3].find_all('span')[0].text.replace(".", "")

    tds = tables[7].find_all('tr')[1].find_all('td')
    res[6] = tds[1].text
    res[7] = tds[3].find_all('span')[0].text.replace(".", "")

    tds = tables[9].find_all('tr')[1].find_all('td')
    res[8] = tds[1].text
    res[9] = tds[3].find_all('span')[0].text.replace(".", "")

    tds = tables[12].find_all('tr')[1].find_all('td')
    res[10] = tds[1].text
    res[11] = tds[3].find_all('span')[0].text.replace(".", "")

    return res

In [ ]:
verbs_df = pd.read_excel("IrrVerbs.xlsx")
verbs_df.drop_duplicates(inplace=True)
verbs_df.dropna(inplace=True)
cols = ['Infinitif', 'Infinitif (pron)', 
        'Indicatif Présent 1', 'Indicatif Présent 1 (pron)',
        'Indicatif Présent 2', 'Indicatif Présent 2 (pron)',
        'Passé simple', 'Passé simple (pron)',
        'Futur simple', 'Futur simple (pron)',
        'Subjonctif Présent', 'Subjonctif Présent (pron)',
        'Participe Passé', 'Participe Passé (pron)']
df = pd.DataFrame(columns=cols)
for word in verbs_df['Infinitif']:
    new_row = dict(zip(cols, parser(word)))
    df = df.append(new_row, ignore_index=True)
df.to_excel("IrrVerbs(parsing).xlsx", index=False)